In [ ]:
import sys
import os
import gc
import torch
from ultralytics import YOLO

# ======================================================================================
# [1] 환경 설정 (이전과 동일하게 통제)
# ======================================================================================
# 데이터셋 경로
data_yaml_path = "C:/Users/hyun8/Desktop/Arakon/detection-base-6/data.yaml"

# 결과 저장 폴더
save_dir_name = "baseline_yolov8m_800_controlled"

print("🚀 [비교군 학습] Standard YOLOv8m (CBAM X, 800px, 동일 환경)")

# ======================================================================================
# [2] 순정 모델 로드 (변인 통제: CBAM 없는 순정 모델 사용)
# ======================================================================================
# 우리가 만든 YAML을 쓰지 않고, ultralytics 기본 모델을 불러옵니다.
model = YOLO('yolov8m.pt') 

# 메모리 정리
gc.collect()
torch.cuda.empty_cache()

# ======================================================================================
# [3] 학습 시작 (변인 통제: 하이퍼파라미터 동일하게 설정)
# ======================================================================================
try:
    model.train(
        data=data_yaml_path,
        
        # [통제 변인 1] 해상도와 배치 사이즈를 우리 모델과 똑같이 맞춤
        imgsz=800,       
        batch=4,         
        
        # [통제 변인 2] 학습률과 옵티마이저를 우리 모델(Finetuning 단계)과 똑같이 맞춤
        epochs=100,      # 충분히 비교될 수 있도록 100회 (조기종료 있음)
        patience=20,     
        optimizer='AdamW', 
        lr0=0.0005,      # 우리가 800 학습할 때 썼던 학습률
        lrf=0.01, 
        cos_lr=True,
        
        # [통제 변인 3] 증강 기법 동일 적용
        mosaic=1.0, 
        mixup=0.1, 
        close_mosaic=10,
        
        # [차이점] 우리는 cls=2.0을 썼지만, 순정 모델은 기본값(0.5)을 사용하여
        # '우리의 튜닝 전략'이 얼마나 유효했는지 비교합니다.
        # (만약 이것까지 똑같이 하려면 cls=2.0을 넣으셔도 됩니다.)
        
        # 시스템 설정
        device=0,
        workers=0,       # 윈도우 필수
        cache=False,     # 메모리 안전
        name=save_dir_name,
        exist_ok=True
    )
    print("\n✅ Baseline 모델 학습 완료!")

except Exception as e:
    print(f"🚨 학습 중 오류 발생: {e}")

# ======================================================================================
# [4] 최종 성능 비교 (변인 통제: TTA 적용 평가)
# ======================================================================================
print("\n📊 [비교 성적표] Baseline (Standard YOLO) 성능 측정 중...")

# 메모리 재정리
gc.collect()
torch.cuda.empty_cache()

try:
    # 학습이 끝난 Baseline 모델 로드
    best_baseline = YOLO(f"runs/detect/{save_dir_name}/weights/best.pt")
    
    # [핵심] 우리 모델과 똑같이 'TTA(augment=True)'를 켜고 공정하게 붙어봅니다.
    metrics = best_baseline.val(
        data=data_yaml_path,
        split='test',    # test 셋 사용
        imgsz=800,
        batch=4,
        augment=True,    # [중요] 우리 모델도 이걸 켰으므로 여기도 켭니다.
        workers=0
    )

    print("\n" + "="*40)
    print("📉 [Baseline] Standard YOLOv8m (800px)")
    print(f"   - mAP@50    : {metrics.box.map50:.4f}")
    print(f"   - mAP@50-95 : {metrics.box.map:.4f}")
    print("-" * 40)
    print("🏆 [Our Model] YOLOv8m + CBAM + Tuning")
    print("   - mAP@50    : 0.9349 (Reference)")
    print("   - mAP@50-95 : 0.6947 (Reference)")
    print("="*40)
    
    # 격차 계산
    gap_50 = 0.9349 - metrics.box.map50
    gap_95 = 0.6947 - metrics.box.map
    
    print(f"💡 결론: 우리 모델이 Baseline보다")
    print(f"   mAP@50은 {gap_50*100:.2f}%p, mAP@50-95는 {gap_95*100:.2f}%p 더 우수합니다!")

except Exception as e:
    print(f"🚨 평가 중 오류 발생: {e}")